In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import column_or_1d

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [2]:
#data = pd.read_csv('training_set_features (1).csv')
import os
os.path.abspath("")

data = pd.read_csv('/home/u184798/WiBD-DataScienceVarsity2023/Standard Deviations/training_set_features (1).csv')


In [3]:
#data = pd.read_csv('training_set_labels (1).csv')
import os
os.path.abspath("")

labels = pd.read_csv('/home/u184798/WiBD-DataScienceVarsity2023/Standard Deviations/training_set_labels (1).csv')

In [4]:
#data = pd.read_csv('training_set_labels (1).csv')
import os
os.path.abspath("")

test = pd.read_csv('/home/u184798/WiBD-DataScienceVarsity2023/Standard Deviations/test_set_features (1).csv')

In [7]:
class CustomOrderLabelEncoder(LabelEncoder):
  def fit(self, y):
    y = column_or_1d(y, warn=True)
    self.classes_ = pd.Series(y).unique()
    return self

In [8]:
data.count()

respondent_id                  26707
h1n1_concern                   26615
h1n1_knowledge                 26591
behavioral_antiviral_meds      26636
behavioral_avoidance           26499
behavioral_face_mask           26688
behavioral_wash_hands          26665
behavioral_large_gatherings    26620
behavioral_outside_home        26625
behavioral_touch_face          26579
doctor_recc_h1n1               24547
doctor_recc_seasonal           24547
chronic_med_condition          25736
child_under_6_months           25887
health_worker                  25903
health_insurance               14433
opinion_h1n1_vacc_effective    26316
opinion_h1n1_risk              26319
opinion_h1n1_sick_from_vacc    26312
opinion_seas_vacc_effective    26245
opinion_seas_risk              26193
opinion_seas_sick_from_vacc    26170
age_group                      26707
education                      25300
race                           26707
sex                            26707
income_poverty                 22284
m

In [9]:
data.values

array([[0, 1.0, 0.0, ..., 0.0, nan, nan],
       [1, 3.0, 2.0, ..., 0.0, 'pxcmvdjn', 'xgwztkwe'],
       [2, 1.0, 1.0, ..., 0.0, 'rucpziij', 'xtkaffoo'],
       ...,
       [26704, 2.0, 2.0, ..., 0.0, nan, nan],
       [26705, 1.0, 1.0, ..., 0.0, 'fcxhlnwr', 'haliazsg'],
       [26706, 0.0, 0.0, ..., 0.0, nan, nan]], dtype=object)

In [10]:
for (columnName, columnData) in data.items():
    print('Column Name : ', columnName, columnData.dtype)
    print('Unique Contents : ', columnData.unique(), "\n")

Column Name :  respondent_id int64
Unique Contents :  [    0     1     2 ... 26704 26705 26706] 

Column Name :  h1n1_concern float64
Unique Contents :  [ 1.  3.  2.  0. nan] 

Column Name :  h1n1_knowledge float64
Unique Contents :  [ 0.  2.  1. nan] 

Column Name :  behavioral_antiviral_meds float64
Unique Contents :  [ 0.  1. nan] 

Column Name :  behavioral_avoidance float64
Unique Contents :  [ 0.  1. nan] 

Column Name :  behavioral_face_mask float64
Unique Contents :  [ 0.  1. nan] 

Column Name :  behavioral_wash_hands float64
Unique Contents :  [ 0.  1. nan] 

Column Name :  behavioral_large_gatherings float64
Unique Contents :  [ 0.  1. nan] 

Column Name :  behavioral_outside_home float64
Unique Contents :  [ 1.  0. nan] 

Column Name :  behavioral_touch_face float64
Unique Contents :  [ 1.  0. nan] 

Column Name :  doctor_recc_h1n1 float64
Unique Contents :  [ 0. nan  1.] 

Column Name :  doctor_recc_seasonal float64
Unique Contents :  [ 0. nan  1.] 

Column Name :  chronic

In [11]:
le_age_group = CustomOrderLabelEncoder()

le_age_group.fit(['18 - 34 Years', '35 - 44 Years', '45 - 54 Years', '55 - 64 Years', '65+ Years'])

 

data['enc_age_group'] = le_age_group.transform(data['age_group'])

 

data.loc[1:10, ['enc_age_group', 'age_group']]

,enc_age_group,age_group
1,1,35 - 44 Years
2,0,18 - 34 Years
3,4,65+ Years
4,2,45 - 54 Years
5,4,65+ Years
6,3,55 - 64 Years
7,2,45 - 54 Years
8,2,45 - 54 Years
9,3,55 - 64 Years
10,2,45 - 54 Years


In [12]:
le_education = CustomOrderLabelEncoder()

le_education.fit(['< 12 Years', '12 Years', 'Some College', 'College Graduate'])

 

data['enc_education'] = pd.Series(

    le_education.transform(data['education'][data['education'].notnull()]),

    index=data['education'][data['education'].notnull()].index

)

 

data.loc[1:10, ['enc_education', 'education']]

,enc_education,education
1,1.0,12 Years
2,3.0,College Graduate
3,1.0,12 Years
4,2.0,Some College
5,1.0,12 Years
6,0.0,< 12 Years
7,2.0,Some College
8,3.0,College Graduate
9,1.0,12 Years
10,1.0,12 Years


In [13]:
le_income_poverty = CustomOrderLabelEncoder()

le_income_poverty.fit(['Below Poverty', '<= $75,000, Above Poverty', '> $75,000'])

 

data['enc_income_poverty'] = pd.Series(
    
    le_income_poverty.transform(data['income_poverty'][data['income_poverty'].notnull()]),
    
    index=data['income_poverty'][data['income_poverty'].notnull()].index
               
)

 

data.loc[1:10, ['enc_income_poverty', 'income_poverty']]

,enc_income_poverty,income_poverty
1,0.0,Below Poverty
2,1.0,"<= $75,000, Above Poverty"
3,0.0,Below Poverty
4,1.0,"<= $75,000, Above Poverty"
5,1.0,"<= $75,000, Above Poverty"
6,1.0,"<= $75,000, Above Poverty"
7,1.0,"<= $75,000, Above Poverty"
8,2.0,"> $75,000"
9,1.0,"<= $75,000, Above Poverty"
10,1.0,"<= $75,000, Above Poverty"


In [14]:
data = data.drop(columns=['race_White', 'race_Black', 'race_Other or Multiple', 'race_Hispanic'], errors='ignore')

data = pd.concat([data, pd.get_dummies(data.race, prefix='race')], axis=1)

 

data.loc[4107:4111, ['race', 'race_White', 'race_Black', 'race_Other or Multiple', 'race_Hispanic']]

,race,race_White,race_Black,race_Other or Multiple,race_Hispanic
4107,White,1,0,0,0
4108,Black,0,1,0,0
4109,White,1,0,0,0
4110,Black,0,1,0,0
4111,White,1,0,0,0


In [15]:
le_marital_status = CustomOrderLabelEncoder()

le_marital_status.fit(['Not Married', 'Married'])

 

data['enc_marital_status'] = pd.Series(
    
    le_marital_status.transform(data['marital_status'][data['marital_status'].notnull()]),
    
    index=data['marital_status'][data['marital_status'].notnull()].index
               
)

 

data.loc[1:10, ['enc_marital_status', 'marital_status']]

,enc_marital_status,marital_status
1,0.0,Not Married
2,0.0,Not Married
3,0.0,Not Married
4,1.0,Married
5,1.0,Married
6,0.0,Not Married
7,1.0,Married
8,1.0,Married
9,0.0,Not Married
10,1.0,Married


In [16]:
le_rent_or_own = CustomOrderLabelEncoder()

le_rent_or_own.fit(['Own', 'Rent'])

 

data['enc_rent_or_own'] = pd.Series(
    
    le_rent_or_own.transform(data['rent_or_own'][data['rent_or_own'].notnull()]),
    
    index=data['rent_or_own'][data['rent_or_own'].notnull()].index
               
)

 

data.loc[1:10, ['enc_rent_or_own', 'rent_or_own']]

,enc_rent_or_own,rent_or_own
1,1.0,Rent
2,0.0,Own
3,1.0,Rent
4,0.0,Own
5,0.0,Own
6,0.0,Own
7,0.0,Own
8,0.0,Own
9,0.0,Own
10,1.0,Rent


In [17]:
le_employment_status = CustomOrderLabelEncoder()

le_employment_status.fit(['Not in Labor Force', 'Employed', 'Unemployed'])

 

data['enc_employment_status'] = pd.Series(
    
    le_employment_status.transform(data['employment_status'][data['employment_status'].notnull()]),
    
    index=data['employment_status'][data['employment_status'].notnull()].index
               
)

 

data.loc[1:10, ['enc_employment_status', 'employment_status']]

,enc_employment_status,employment_status
1,1.0,Employed
2,1.0,Employed
3,0.0,Not in Labor Force
4,1.0,Employed
5,1.0,Employed
6,1.0,Employed
7,1.0,Employed
8,1.0,Employed
9,0.0,Not in Labor Force
10,1.0,Employed


In [18]:
data = data.drop(columns=['region_oxchjgsf', 'region_bhuqouqj', 'region_qufhixun', 'region_lrircsnp', 'region_atmpeygn', 'region_lzgpxyit',
 'region_fpwskwrf', 'region_mlyzmhmf', 'region_dqpwygqj', 'region_kbazzjca'], errors='ignore')

data = pd.concat([data, pd.get_dummies(data.hhs_geo_region, prefix='region')], axis=1)

 

data.loc[4107:4111, ['hhs_geo_region', 'region_oxchjgsf', 'region_bhuqouqj', 'region_qufhixun', 'region_lrircsnp', 'region_atmpeygn', 'region_lzgpxyit',
 'region_fpwskwrf', 'region_mlyzmhmf', 'region_dqpwygqj', 'region_kbazzjca']]

,hhs_geo_region,region_oxchjgsf,region_bhuqouqj,region_qufhixun,region_lrircsnp,region_atmpeygn,region_lzgpxyit,region_fpwskwrf,region_mlyzmhmf,region_dqpwygqj,region_kbazzjca
4107,lzgpxyit,0,0,0,0,0,1,0,0,0,0
4108,qufhixun,0,0,1,0,0,0,0,0,0,0
4109,kbazzjca,0,0,0,0,0,0,0,0,0,1
4110,lzgpxyit,0,0,0,0,0,1,0,0,0,0
4111,bhuqouqj,0,1,0,0,0,0,0,0,0,0


In [19]:
data = data.drop(columns=['msa_Non-MSA', 'msa_MSA, Not Principle  City', 'msa_MSA, Principle City'], errors='ignore')

data = pd.concat([data, pd.get_dummies(data.census_msa, prefix='msa')], axis=1)

 

data.loc[4107:4111, ['census_msa', 'msa_Non-MSA', 'msa_MSA, Not Principle  City', 'msa_MSA, Principle City']]

,census_msa,msa_Non-MSA,"msa_MSA, Not Principle City","msa_MSA, Principle City"
4107,"MSA, Not Principle City",0,1,0
4108,"MSA, Not Principle City",0,1,0
4109,"MSA, Principle City",0,0,1
4110,"MSA, Principle City",0,0,1
4111,"MSA, Principle City",0,0,1


In [20]:
data = data.drop(columns=['pxcmvdjn', 'rucpziij', 'wxleyezf', 'saaquncn', 'xicduogh', 'ldnlellj',
 'wlfvacwt', 'nduyfdeo', 'fcxhlnwr', 'vjjrobsf', 'arjwrbjb', 'atmlpfrs',
 'msuufmds', 'xqicxuve', 'phxvnwax', 'dotnnunm', 'mfikgejo', 'cfqqtusy',
 'mcubkhph', 'haxffmxo', 'qnlwzans'], errors='ignore')

data = pd.concat([data, pd.get_dummies(data.employment_industry, prefix='industry')], axis=1)

 

data.loc[4107:4111, ['employment_industry', 'industry_pxcmvdjn', 'industry_rucpziij', 'industry_wxleyezf', 'industry_saaquncn', 'industry_xicduogh', 'industry_ldnlellj',
 'industry_wlfvacwt', 'industry_nduyfdeo', 'industry_fcxhlnwr', 'industry_vjjrobsf', 'industry_arjwrbjb', 'industry_atmlpfrs',
 'industry_msuufmds', 'industry_xqicxuve', 'industry_phxvnwax', 'industry_dotnnunm', 'industry_mfikgejo', 'industry_cfqqtusy',
 'industry_mcubkhph', 'industry_haxffmxo', 'industry_qnlwzans']]

,employment_industry,industry_pxcmvdjn,industry_rucpziij,industry_wxleyezf,industry_saaquncn,industry_xicduogh,industry_ldnlellj,industry_wlfvacwt,industry_nduyfdeo,industry_fcxhlnwr,...,industry_atmlpfrs,industry_msuufmds,industry_xqicxuve,industry_phxvnwax,industry_dotnnunm,industry_mfikgejo,industry_cfqqtusy,industry_mcubkhph,industry_haxffmxo,industry_qnlwzans
4107,NaN,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4108,wxleyezf,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4109,NaN,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4110,NaN,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4111,rucpziij,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
data = data.drop(columns=['xgwztkwe', 'xtkaffoo', 'emcorrxb', 'vlluhbov', 'xqwwgdyp', 'ccgxvspp',
 'qxajmpny', 'kldqjyjy', 'mxkfnird', 'hfxkjkmi', 'bxpfxfdn', 'ukymxvdu',
 'cmhcxjea', 'haliazsg', 'dlvbwzss', 'xzmlyyjv', 'oijqvulv', 'rcertsgn',
 'tfqavkke', 'hodpvpew', 'uqqtjvyb', 'pvmttkik', 'dcjcmpih'], errors='ignore')

data = pd.concat([data, pd.get_dummies(data.employment_occupation, prefix='occupation')], axis=1)

 

data.loc[4107:4111, ['employment_occupation', 'occupation_xgwztkwe', 'occupation_xtkaffoo', 'occupation_emcorrxb', 'occupation_vlluhbov', 'occupation_xqwwgdyp', 'occupation_ccgxvspp',
 'occupation_qxajmpny', 'occupation_kldqjyjy', 'occupation_mxkfnird', 'occupation_hfxkjkmi', 'occupation_bxpfxfdn', 'occupation_ukymxvdu',
 'occupation_cmhcxjea', 'occupation_haliazsg', 'occupation_dlvbwzss', 'occupation_xzmlyyjv', 'occupation_oijqvulv', 'occupation_rcertsgn',
 'occupation_tfqavkke', 'occupation_hodpvpew', 'occupation_uqqtjvyb', 'occupation_pvmttkik', 'occupation_dcjcmpih']]

,employment_occupation,occupation_xgwztkwe,occupation_xtkaffoo,occupation_emcorrxb,occupation_vlluhbov,occupation_xqwwgdyp,occupation_ccgxvspp,occupation_qxajmpny,occupation_kldqjyjy,occupation_mxkfnird,...,occupation_haliazsg,occupation_dlvbwzss,occupation_xzmlyyjv,occupation_oijqvulv,occupation_rcertsgn,occupation_tfqavkke,occupation_hodpvpew,occupation_uqqtjvyb,occupation_pvmttkik,occupation_dcjcmpih
4107,NaN,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4108,emcorrxb,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4109,NaN,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4110,NaN,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4111,tfqavkke,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [22]:
# get indices where any feature has a NaN value

index_nans = data[data.isnull().any(axis=1)].index

# drop those rows from our feature dataframe and label dataframe

nonan_data = data.drop(index_nans)
nonan_labels = labels.drop(index_nans)

nonan_data.count()

respondent_id                6437
h1n1_concern                 6437
h1n1_knowledge               6437
behavioral_antiviral_meds    6437
behavioral_avoidance         6437
                             ... 
occupation_vlluhbov          6437
occupation_xgwztkwe          6437
occupation_xqwwgdyp          6437
occupation_xtkaffoo          6437
occupation_xzmlyyjv          6437
Length: 103, dtype: int64